In [ ]:
import numpy as np
import librosa
import soundfile as sf
from pydub import AudioSegment
from pydub.effects import strip_silence
import random
import os
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

def apply_audio_augmentation(audio, sample_rate, augmentation_type):
    # Load the audio file
    if augmentation_type == "time_shift":
        augmented_audio = np.roll(y, 3000)
    elif augmentation_type == "speed_change":
        rate = random.uniform(0.7, 1.3)
        augmented_audio = librosa.effects.time_stretch(y, rate=rate)
    elif augmentation_type == "pitch_shift":
        augmented_audio = librosa.effects.pitch_shift(y, sr = sr, n_steps=random.uniform(-2, 2))
    elif augmentation_type == "noise_injection":
        # Add white noise
        noise_factor = 0.005
        white_noise = np.random.randn(len(y)) * noise_factor
        augmented_audio = y + white_noise

    return augmented_audio

#trial use case, upload whatever audio file, download untitled to see what has been changed
#apply_audio_augmentation('/content/car_horn_5.wav','/content/untitled.wav',"changing_speed")

In [ ]:
def augment_audio(input_path, output_path, max_samples=640):
    """
    Augments audio files in the input directory and saves them to the output directory.
    Only augments folders with less than max_samples files.
    """
    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)

    # Loop through each class folder in the input directory
    for class_folder in os.listdir(input_path):
        class_path = os.path.join(input_path, class_folder)
        if os.path.isdir(class_path):
            # Count the number of files in the class folder
            num_files = len([file for file in os.listdir(class_path) if file.endswith('.wav')])

            # If the number of files is less than max_samples, augment the files
            if num_files < max_samples:
                print(f"Augmenting files in folder: {class_folder}")
                # Loop through each file in the class folder
                for file_name in os.listdir(class_path):
                    if file_name.endswith(".wav"):
                        # Load the audio file
                        audio_path = os.path.join(class_path, file_name)
                        y, sr = librosa.load(audio_path, sr=None)

                        # Choose a random augmentation type
                        augmentation_type = random.choice(["time_shift", "speed_change", "pitch_shift", "noise_injection"])

                        # Apply the chosen augmentation
                        augmented_audio = apply_audio_augmentation(y, sr, augmentation_type)

                        # Save the augmented audio to the output directory
                        output_file_name = "aug_" + file_name

                        output_class_folder = os.path.join(output_path, class_folder)
                        os.makedirs(output_class_folder, exist_ok=True)
                        output_audio_path = os.path.join(output_class_folder, output_file_name)

                        sf.write(output_audio_path, augmented_audio, sr)
                        print(f"Augmented file saved: {output_audio_path}")
            else:
                print(f"Skipping folder: {class_folder} (already has {num_files} samples)")



In [ ]:
input_folder = "/content/drive/My Drive/audio_dataset/audio_dataset/train"
output_folder = "/content/drive/My Drive/audio_dataset/audio_dataset/train_augmented"

augment_audio(input_folder, output_folder)